In [ ]:
import requests
import pandas as pd

link = "https://servicodados.ibge.gov.br/api/v3/agregados/7392/periodos/-6/variaveis/10484?localidades=N1[all]"

requisicao = requests.get(link)
dados = requisicao.json()

# Extrair os dados relevantes
dados_organizados = []

for item in dados:
    id_variavel = item['id']
    variavel = item['variavel']
    unidade = item['unidade']
    
    for resultado in item['resultados']:
        for classificacao in resultado['classificacoes']:
            id_classificacao = classificacao['id']
            nome_classificacao = classificacao['nome']
            categoria_id, categoria_nome = list(classificacao['categoria'].items())[0]
            
            for serie in resultado['series']:
                localidade_id = serie['localidade']['id']
                localidade_nome = serie['localidade']['nome']
                nivel_id = serie['localidade']['nivel']['id']
                nivel_nome = serie['localidade']['nivel']['nome']
                
                for ano, valor in serie['serie'].items():
                    dados_organizados.append({
                        'ID Variável': id_variavel,
                        'Variável': variavel,
                        'Unidade': unidade,
                        'ID Classificação': id_classificacao,
                        'Classificação': nome_classificacao,
                        'ID Categoria': categoria_id,
                        'Categoria': categoria_nome,
                        'ID Localidade': localidade_id,
                        'Localidade': localidade_nome,
                        'ID Nível': nivel_id,
                        'Nível': nivel_nome,
                        'Ano': ano,
                        'Valor': valor
                    })

# Criar o DataFrame
df = pd.DataFrame(dados_organizados)

df.to_excel(r"C:\Users\marce\OneDrive\Arquivos Antigos\Documentos\GitHub\Python_Projetos\Datasets\teste.xlsx")

# Exibir o DataFrame
print(df)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import random
import pyodbc
from datetime import date
from faker import Faker

fake = Faker() #variavel para usar Faker

data1 = fake.date_time_between(start_date='-15y',end_date='now')

print(data1)

[0.12 ,0.099,0.07,0.065,0.07,0.014,0.012,0.081,0.011,0.099,0.081,0.012,0.012,0.011,0.012,0.018,0.009,0.009,0.008,0.008,0.017,0.006,0.006,0.017,0.015,0.014,0.004,0.003,0.007,0.009,0.019,0.002,0.007,0.011,0.013,0.004,0.004,0.003,0.008,0.01]

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]


2021-03-26 04:53:34


In [ ]:
import pandas as pd

# Exemplo de DataFrame com nomes
data = {
    "NOME": ["João", "João", "João", "João", "João", "Maria", "Maria", "Maria", "Pedro", "Ferna", "Ferna", "Ferna", "Rober", "Rober", "Rober", "Rober", "Rober", "Rober"]
}
df = pd.DataFrame(data)

# Exemplo de DataFrame com idades
data1 = {
    "IDADE": [
        25, 25, 25, 25,  # Sequência de João
        25, 25, 25,       # Sequência de Maria
        27,               # Sequência de Pedro
        37, 37, 37,       # Sequência de Ferna
        25, 25, 25, 25, 25, 25  # Sequência de Rober
    ]
}
df1 = pd.DataFrame(data1)

# Lista para armazenar as primeiras idades de cada sequência
primeiras_idades = []

# Índice inicial para percorrer as idades
indice = 0

# Função para agrupar nomes em sequências
def agrupar_sequencias_nomes(nomes):
    sequencias = []
    if len(nomes) == 0:
        return sequencias
    
    nome_atual = nomes[0]
    inicio = 0
    
    for i, nome in enumerate(nomes):
        if nome != nome_atual:
            sequencias.append((nome_atual, inicio, i - 1))  # (nome, início, fim)
            nome_atual = nome
            inicio = i
    sequencias.append((nome_atual, inicio, len(nomes) - 1))  # Última sequência
    return sequencias

# Agrupar os nomes em sequências
sequencias_nomes = agrupar_sequencias_nomes(df["NOME"].tolist())

# Verificar se o número de sequências de nomes corresponde ao número de sequências de idades
if len(sequencias_nomes) != len(df1["IDADE"].unique()):
    raise ValueError("O número de sequências de nomes não corresponde ao número de sequências de idades.")

# Associar cada nome à primeira idade de sua sequência
for nome, inicio, fim in sequencias_nomes:
    # Adicionar a primeira idade da sequência correspondente ao nome
    primeiras_idades.append(df1["IDADE"][indice])
    
    # Avançar o índice para o início da próxima sequência de idades
    indice += (fim - inicio + 1)

# Criar o DataFrame resultante
df_resultante = pd.DataFrame({
    "NOME": [nome for nome, _, _ in sequencias_nomes],  # Nomes únicos
    "IDADE": primeiras_idades
})

print("DataFrame resultante:")
print(df_resultante)

In [12]:
#Script que gera NF, ITEMNF e VENDAS em formaato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

# Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 3001
quant_maxima_itens = 100
variavel_controle_maximo_LOOP = 1300 #Valor maximo que da nota inicial dentro do loop de geracao de NF's

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

total = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()

print(total)

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
153


In [1]:
#Gerando NF, ITEMNF e VENDAS em formato .xlsx na pasta

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
from sqlalchemy.exc import SQLAlchemyError

# Configurações iniciais
Tamanho_do_pacote_de_notas = 1000
NUMERO_INICIAL_NOTA = 1000
VARIAVEL_CONTROLE_MAXIMO_LOOP = 2000
QUANT_MAXIMA_ITENS = 10

fake = Faker()

# Configurações do banco de dados
SERVER = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
BANCO_DADOS = "MERCADO_OLTP"
DRIVER = "ODBC Driver 17 for SQL Server"
CONN_STRING = f"mssql+pyodbc://{SERVER}/{BANCO_DADOS}?driver={DRIVER}&Trusted_Connection=yes"
engine = create_engine(CONN_STRING, pool_size=10, max_overflow=30)

# Função para testar a conexão
def testar_conexao():
    try:
        with engine.connect() as conexao:
            conexao.execute(text("SELECT 1"))
            print("Conexão com o banco de dados estabelecida com sucesso.")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False

# Função para obter o total de registros em uma tabela
try:
    with engine.connect() as conexao:
        valores_id_nf = 1000 + conexao.execute(text(f"SELECT COUNT(IDNOTAFISCAL) FROM NOTAFISCAL")).scalar()
except Exception as e:
    print(f'Erro ao contas os id da nfs:{e}')   

NUMERO_INICIAL_NOTA = valores_id_nf
VARIAVEL_CONTROLE_MAXIMO_LOOP = valores_id_nf + Tamanho_do_pacote_de_notas

# Função para gerar uma nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, data_nota):
    quant_itens = random.randint(1, QUANT_MAXIMA_ITENS)
    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, obter_total("CLIENTE", "IDCLIENTE"))
    id_loja = random.randint(1, obter_total("LOJA", "IDLOJA"))
    id_cfop = random.randint(1, 6)
    chave_acesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    versao = 1

    itens = []
    for _ in range(quant_itens):
        produto = random.choice(lista_produtos)
        preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
        id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
        quantidade = random.randint(1, 10)
        total_item = round(quantidade * preco_unit, 2)
        itens.append((data_nota, numero_nota, produto, quantidade, preco_unit, total_item, 
                      id_forma_pgmt, id_cliente, id_loja, id_cfop, chave_acesso, data_criacao, 
                      "", versao, id_produto))

    total_nota = round(sum(item[5] for item in itens), 2)
    itens = [item + (total_nota,) for item in itens]

    df_nota_fiscal = pd.DataFrame(
        itens, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", 
                        "TOTAL_ITEM", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                        "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO", "IDPRODUTO", "TOTAL_NOTA"]
    )

    df_nota_fiscal_2 = pd.DataFrame(
        [(data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chave_acesso, data_criacao, 
          "", versao, total_nota)],
        columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                 "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO", "TOTAL_NOTA"]
    )

    return df_nota_fiscal, df_nota_fiscal_2

# Função para obter o total de registros em uma tabela
def obter_total(tabela, coluna):
    with engine.connect() as conexao:
        return conexao.execute(text(f"SELECT COUNT({coluna}) FROM {tabela}")).scalar()

# Função para salvar DataFrames em arquivos Excel
def salvar_dataframes(dataframes, pastas, numero_inicial_nota, numero_maximo_nota):
    id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
    for nome_df, df in dataframes.items():
        try:
            nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"
            caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)
            df.to_excel(caminho_completo, index=False)
        except Exception as e:
            print(f"Erro ao salvar o DataFrame {nome_df}: {e}")

# Caminhos das pastas
PASTAS = {
    'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
    'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
    'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
}

# Execução principal
if testar_conexao():
    numero_inicial_nota = NUMERO_INICIAL_NOTA
    while numero_inicial_nota <= VARIAVEL_CONTROLE_MAXIMO_LOOP:
        try:
            incremento = random.randint(50, 85)
            numero_maximo_nota = numero_inicial_nota + incremento
            if numero_maximo_nota <= VARIAVEL_CONTROLE_MAXIMO_LOOP:
                df_prod = pd.read_sql(text("SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO"), engine)
                lista_produtos = df_prod['NOME'].to_list()

                df_notas_fiscais = []
                df_notas_fiscais_2 = []
                for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
                    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
                    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
                    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
                    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
                    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
                    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_produtos, data_nota)
                    df_notas_fiscais.append(df_nota_fiscal)
                    df_notas_fiscais_2.append(df_2)

                df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
                df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

                df_ITEMNOTA = df_todas_notas[['Numero_Nota', 'IDPRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_ITEMNOTA.columns = ['ID_NOTAFISCAL', 'ID_PRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_NOTAFISCAL = df_todas_notas2[['DATA_NOTAFISCAL', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_NOTAFISCAL.columns = ['DATA', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_VENDA = df_todas_notas[['DATA_NOTAFISCAL', 'Numero_Nota', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_NOTA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_VENDA.columns = ['DATA', 'ID_NOTAFISCAL', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_VENDA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                dataframes = {
                    'ITEMNOTA': df_ITEMNOTA,
                    'NOTAFISCAL': df_NOTAFISCAL,
                    'VENDA': df_VENDA
                }

                salvar_dataframes(dataframes, PASTAS, numero_inicial_nota, numero_maximo_nota)
                print(f"Arquivos de vendas de {numero_inicial_nota} até {numero_maximo_nota} salvos com sucesso.")
                numero_inicial_nota = numero_maximo_nota + 1
            else:
                numero_maximo_nota = VARIAVEL_CONTROLE_MAXIMO_LOOP - 1
                df_prod = pd.read_sql(text("SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO"), engine)
                lista_produtos = df_prod['NOME'].to_list()

                df_notas_fiscais = []
                df_notas_fiscais_2 = []
                for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
                    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
                    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
                    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
                    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
                    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
                    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_produtos, data_nota)
                    df_notas_fiscais.append(df_nota_fiscal)
                    df_notas_fiscais_2.append(df_2)

                if not df_notas_fiscais or not df_notas_fiscais_2:
                    print(f"Erro: Nenhuma nota fiscal gerada para o intervalo {numero_inicial_nota} até {numero_maximo_nota}.")
                    break

                df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
                df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

                df_ITEMNOTA = df_todas_notas[['Numero_Nota', 'IDPRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_ITEMNOTA.columns = ['ID_NOTAFISCAL', 'ID_PRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_NOTAFISCAL = df_todas_notas2[['DATA_NOTAFISCAL', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_NOTAFISCAL.columns = ['DATA', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_VENDA = df_todas_notas[['DATA_NOTAFISCAL', 'Numero_Nota', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_NOTA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_VENDA.columns = ['DATA', 'ID_NOTAFISCAL', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_VENDA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                dataframes = {
                    'ITEMNOTA': df_ITEMNOTA,
                    'NOTAFISCAL': df_NOTAFISCAL,
                    'VENDA': df_VENDA
                }

                salvar_dataframes(dataframes, PASTAS, numero_inicial_nota, numero_maximo_nota)
                print(f"Arquivos de vendas de {numero_inicial_nota} até {numero_maximo_nota} salvos com sucesso.")
                numero_inicial_nota = numero_maximo_nota + 5
        except Exception as e:
            print(f"Erro ao gerar os arquivos: {e}")

#Lendo os aruivos na pasta e enviando para o banco de dados

try:
    with engine.connect() as conn:
            conn.execute(text("ALTER TABLE ITEMNOTA DROP CONSTRAINT FK_ITEMNOTA_NOTAFISCAL"))
            conn.commit()
            print(f"FK atualizada com sucesso!")
except Exception as e:
    print(f"Erro ao executar: {e}")

def ler_arquivos_xlsx(caminho_pasta):
    dataframes = []
    for arquivo in os.listdir(caminho_pasta):
        if arquivo.endswith(".xlsx"):
            try:
                caminho_completo = os.path.join(caminho_pasta, arquivo)
                df = pd.read_excel(caminho_completo)
                dataframes.append(df)
                #print(f"Arquivo {arquivo} lido com sucesso.")
            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {e}")
    
    print(f"Arquivos de {caminho_pasta} lidos com sucesso.")
    return pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

#Caminhos das pastas
caminho_itemnota = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\ITEMNOTA"
caminho_notafiscal = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\NOTAFISCAL"
caminho_venda = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\VENDA"

#Lê os arquivos e cria os DataFrames
df_ITEMNOTA = ler_arquivos_xlsx(caminho_itemnota)
df_NOTAFISCAL = ler_arquivos_xlsx(caminho_notafiscal)
df_VENDA = ler_arquivos_xlsx(caminho_venda)

#print("df_ITEMNOTA")
#display(df_ITEMNOTA)
#print("df_NOTAFISCAL")
#display(df_NOTAFISCAL)
#print("df_VENDA")
#display(df_VENDA)

#Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Verifica se a coluna é do tipo datetime
    if np.issubdtype(col.dtype, np.datetime64):
        # Se for datetime, retorna a coluna original sem modificações
        return col
    
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip() #Converte a coluna para string, remove espaços extras e aplica strip
    
    #Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif col.name in ["DATA_CRIACAO", "DATA_ATLZ"]:  # Se for uma coluna de data, Converte valores inválidos (como 'None') para NaT (Not a Time)
        return pd.to_datetime(col_stripped, format='%Y-%m-%d', errors='coerce').dt.date
    else:  # Se for string ou outro tipo
        return col_stripped

tabela = "ITEMNOTA"          #NOME DA TABELA A SEREM ADICIONADOS OS DADOS
coluna = "ID_NOTAFISCAL"       #NOME DA COLUNA DA TABELA QUE O SCRCIPT FARA A VERIFICACAO SE O DADO JA EXISTE NO BANCO DE DADOS
tabela2 = "NOTAFISCAL" 
coluna2 = "CHAVE_ACESSO"    
tabela3 = "VENDA" 
coluna3 = "ID_NOTAFISCAL"

with engine.connect() as conn:
    # Consulta os valores existentes na coluna "NOME" da tabela "PRODUTO"
    query = text(f"SELECT {coluna} FROM {tabela}")
    lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()

    query2 = text(f"SELECT {coluna2} FROM {tabela2}")
    lista_valores_existentes2 = pd.read_sql(query2, conn)[coluna2].tolist()

    query3 = text(f"SELECT {coluna3} FROM {tabela3}")
    lista_valores_existentes3 = pd.read_sql(query3, conn)[coluna3].tolist()

    # Filtra os valores que não existem na tabela
    df_ITEMNOTA_filtrado =   df_ITEMNOTA[~df_ITEMNOTA["ID_NOTAFISCAL"].isin(lista_valores_existentes)]
    df_NOTAFISCAL_filtrado = df_NOTAFISCAL[~df_NOTAFISCAL["CHAVE_ACESSO"].isin(lista_valores_existentes2)]
    df_VENDA_filtrado =      df_VENDA[~df_VENDA["ID_NOTAFISCAL"].isin(lista_valores_existentes3)]

    # Se houver valores para adicionar
    if not df_ITEMNOTA_filtrado.empty or not df_NOTAFISCAL_filtrado.empty or not df_VENDA_filtrado.empty:
        # Limpa os espaços desnecessários
        df_ITEMNOTA_filtrado = df_ITEMNOTA_filtrado.apply(strip_all_types)
        df_NOTAFISCAL_filtrado = df_NOTAFISCAL_filtrado.apply(strip_all_types)
        df_VENDA_filtrado = df_VENDA_filtrado.apply(strip_all_types)

        # Define os tipos de dados para a tabela PRODUTO
        tipo_dados_ITEMNOTA = {
                'ID_NOTAFISCAL':  INTEGER,
                'ID_PRODUTO':     INTEGER,
                'QUANTIDADE':     INTEGER,
                'PRECO_UNIT':     NUMERIC(18,2),
                'TOTAL_ITEM':     NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
        }

        tipo_dados_NOTAFISCAL = {
                'DATA':           DATE,
                'TOTAL_NOTA':     NUMERIC(18,2),
                'ID_FORMA_PGMT':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_CFOP':        INTEGER,
                'CHAVE_ACESSO':   VARCHAR(100),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
        }

        tipo_dados_VENDA = {
                'DATA':           DATE,
                'ID_NOTAFISCAL':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_FORMA_PGMT':  INTEGER,
                'TOTAL_VENDA':    NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
        }

        # Exibe os dados que serão adicionados
        print(F"Notas a serem adicionas adicionadas")
        #display(df_ITEMNOTA_filtrado)
        display(df_NOTAFISCAL_filtrado)
        #display(df_VENDA_filtrado)

        # Adiciona os dados ao banco de dados
        df_ITEMNOTA_filtrado.to_sql(
            name=tabela,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_ITEMNOTA
        )

        df_NOTAFISCAL_filtrado.to_sql(
            name=tabela2,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_NOTAFISCAL
        )
        df_VENDA_filtrado.to_sql(
            name=tabela3,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_VENDA
        )

        print("Produtos inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")

try:
    with engine.connect() as conn:
            conn.execute(text("ALTER TABLE ITEMNOTA ADD CONSTRAINT FK_ITEMNOTA_NOTAFISCAL FOREIGN KEY(ID_NOTAFISCAL) REFERENCES NOTAFISCAL(IDNOTAFISCAL)"))
            conn.commit()
            print(f"FK adicionada com sucesso!")
except Exception as e:
    print(f"Erro ao executar: {e}")


# Fecha a conexão com o banco de dados
engine.dispose()

Conexão com o banco de dados estabelecida com sucesso.
Arquivos de vendas de 2800 até 2865 salvos com sucesso.
Arquivos de vendas de 2866 até 2942 salvos com sucesso.
Arquivos de vendas de 2943 até 3020 salvos com sucesso.
Arquivos de vendas de 3021 até 3084 salvos com sucesso.
Arquivos de vendas de 3085 até 3162 salvos com sucesso.
Arquivos de vendas de 3163 até 3213 salvos com sucesso.
Arquivos de vendas de 3214 até 3288 salvos com sucesso.
Arquivos de vendas de 3289 até 3374 salvos com sucesso.
Arquivos de vendas de 3375 até 3430 salvos com sucesso.
Arquivos de vendas de 3431 até 3510 salvos com sucesso.
Arquivos de vendas de 3511 até 3594 salvos com sucesso.
Arquivos de vendas de 3595 até 3660 salvos com sucesso.
Arquivos de vendas de 3661 até 3724 salvos com sucesso.
Arquivos de vendas de 3725 até 3799 salvos com sucesso.
FK atualizada com sucesso!
Arquivos de C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\ITEMNOTA lidos com sucesso

,DATA,TOTAL_NOTA,ID_FORMA_PGMT,ID_CLIENTE,ID_LOJA,ID_CFOP,CHAVE_ACESSO,DATA_CRIACAO,DATA_ATLZ,VERSAO
1800,2024-01-04 16:57:06,520.06,7,691348,149,5,62442874285785277793652669152154779188839182,2024-01-04,NaN,1
1801,2024-01-15 21:08:46,243.43,5,513228,101,5,17693044113390641663275439090972147506068842,2024-01-15,NaN,1
1802,2024-01-09 11:41:05,839.99,3,919828,56,5,82000479628025231280934787850137981895310730,2024-01-09,NaN,1
1803,2024-01-08 18:06:55,233.36,8,174737,17,2,50544340138179169480880318127706882611241705,2024-01-08,NaN,1
1804,2024-01-02 21:21:14,10.38,8,694942,148,2,17861061564266902695432849383092056119457229,2024-01-02,NaN,1
...,...,...,...,...,...,...,...,...,...,...
2795,2024-01-07 15:27:27,345.21,2,189055,66,1,82958562459680935973067560007394484916162499,2024-01-07,NaN,1
2796,2024-01-06 12:42:13,1160.14,5,212893,17,2,03295397574446047906958975192873656560013699,2024-01-06,NaN,1
2797,2024-01-12 19:56:30,1613.65,1,341605,68,1,16485290362750703544971932133000449722992403,2024-01-12,NaN,1
2798,2024-01-14 17:37:15,5417.16,7,325400,51,6,24001120198621692237768619001636609771086139,2024-01-14,NaN,1


Produtos inseridos com sucesso.
FK adicionada com sucesso!
